In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [2]:
!pip install transformers -q
!pip install datasets -q
!pip install nltk -q
import nltk
nltk.download('stopwords')
!pip install pymorphy2 -q

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.ticker as ticker
from transformers import get_scheduler
import matplotlib.pyplot as plt

import numpy as np

import random
import math
import time
import string
import pymorphy2
import re

# datasets from huggingface
from datasets import load_dataset
from transformers import BertTokenizer, BertModel

from nltk.corpus import stopwords
import nltk

from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

import os
from datetime import datetime
from torch.nn.utils.rnn import pad_sequence

import seaborn as sns

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [26]:
BASE_FOLDER_PATH = 'gdrive/MyDrive/Deep_Learning_MIPT_2/text_classification'
BERT_MODEL_NAME = 'bert-base-cased'
BATCH_SIZE = 32
EPOCHS = 5
MODEL_NAME = 'model_v1'
LR = 0.00005
N_HEADS = 1
# TRAIN_SIZE = 20000
TRAIN_SIZE = 2000
TEST_SIZE = 1000

MODEL_FOLDER_PATH = os.path.join(BASE_FOLDER_PATH, MODEL_NAME)

SCHEDULER_LAMBDA_PARAM = 0.96
PAD_IND = 0
HIDDEN_DIM = 768

ENGLISH_STOP_WORDS = set(stopwords.words('english'))
PUNCT_WORD_TOKENIZER = nltk.WordPunctTokenizer() # for preprocess
# for broadening your horizons use it for lemmatization
MORPH_ANALYZER = pymorphy2.MorphAnalyzer()

In [27]:
os.makedirs(MODEL_FOLDER_PATH, exist_ok=True)
print(MODEL_FOLDER_PATH)

gdrive/MyDrive/Deep_Learning_MIPT_2/text_classification/model_v1


##Train the BERT model for this task

In [28]:
# download pretrained models
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4).to(device)
for w in model.distilbert.parameters():
    w._trainable= False
for w in model.classifier.parameters():
    w._trainable = True

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Prepare dataset

In [29]:
dataset_dict = load_dataset('ag_news')
train_ds = dataset_dict['train']
test_ds = dataset_dict['test']

In [30]:
def tokenize_function(examples, tokenizer=tokenizer):
    return tokenizer(
        examples["text"],
        padding="max_length",
        max_length=512,
        truncation=True,
        return_token_type_ids="token_type_ids" in tokenizer.model_input_names)

def ds_object_process(ds_obj):
  return ds_obj.map(
        tokenize_function, batched=True
      ).remove_columns(["text"]).rename_column("label", "labels")

In [31]:
# Take small train to speed up train
tokenized_train_dataset = ds_object_process(train_ds.select(range(TRAIN_SIZE)))
tokenized_train_dataset.set_format("torch")
tokenized_test_dataset = ds_object_process(test_ds.select(range(TEST_SIZE)))
tokenized_test_dataset.set_format("torch")

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

### Create dataloaders, optimizer and scheduler

In [32]:
train_dataloader = DataLoader(tokenized_train_dataset, shuffle=True, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=BATCH_SIZE)

In [33]:
optimizer = optim.AdamW(model.parameters(), lr=LR)
num_training_steps = EPOCHS * len(train_dataloader)
scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

## Run train loop

In [34]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(EPOCHS):
    train_loss = 0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        train_loss += loss.item()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    train_loss /= len(train_dataloader)
    print(f'EPOCH {epoch}: train_loss={train_loss}')

  0%|          | 0/315 [00:00<?, ?it/s]

EPOCH 0: train_loss=0.6621666820276351
EPOCH 1: train_loss=0.25331717339299975
EPOCH 2: train_loss=0.13938427071959253
EPOCH 3: train_loss=0.07739687240904286
EPOCH 4: train_loss=0.05285098762916667


### Run validation

In [35]:
from datasets import load_metric

metric = load_metric("accuracy")
model.eval()
for batch in tqdm(test_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


  0%|          | 0/32 [00:00<?, ?it/s]

{'accuracy': 0.889}